Imports

In [20]:
from ultralytics import YOLO
import cv2
import pandas as pd

In [21]:
plate_det_model_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/models/YOLOv8/yolov8n_90e_cust/runs/detect/train4/weights/best.pt'
testvideo_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/video_data/testvideo_highway_mp4.mp4'

Load the model

In [22]:
vehicle_det_model=YOLO('yolov8n.pt')
vehicle_ids = [2,3,4,5,6,7,8]
plate_det_model = YOLO(plate_det_model_path)

New class needed for tracking

In [23]:
import numpy as np
from filterpy.kalman import KalmanFilter

def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))
def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))
class KalmanBoxTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4) 
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = []
    self.hits = 0
    self.hit_streak = 0
    self.age = 0
  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))
  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]
  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)

def linear_assignment(cost_matrix):
    try:
        import lap
        _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
        return np.array([[y[i],i] for i in x if i >= 0]) #
    except ImportError:
        from scipy.optimize import linear_sum_assignment
        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))
    
def iou_batch(bb_test, bb_gt):
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_test = np.expand_dims(bb_test, 1)

    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])                                      
     + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                              
    return(o)  

def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
    if(len(trackers)==0):
      return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)
    iou_matrix = iou_batch(detections, trackers)
    if min(iou_matrix.shape) > 0:
      a = (iou_matrix > iou_threshold).astype(np.int32)
      if a.sum(1).max() == 1 and a.sum(0).max() == 1:
          matched_indices = np.stack(np.where(a), axis=1)
      else:
        matched_indices = linear_assignment(-iou_matrix)
    else:
      matched_indices = np.empty(shape=(0,2))
    unmatched_detections = []
    for d, det in enumerate(detections):
      if(d not in matched_indices[:,0]):
        unmatched_detections.append(d)
    unmatched_trackers = []
    for t, trk in enumerate(trackers):
      if(t not in matched_indices[:,1]):
        unmatched_trackers.append(t)
    #filter out matched with low IOU
    matches = []
    for m in matched_indices:
      if(iou_matrix[m[0], m[1]]<iou_threshold):
        unmatched_detections.append(m[0])
        unmatched_trackers.append(m[1])
      else:
        matches.append(m.reshape(1,2))
    if(len(matches)==0):
      matches = np.empty((0,2),dtype=int)
    else:
      matches = np.concatenate(matches,axis=0)
    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


class MotionTracker(object):
    def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0
    
    
    def update(self, dets=np.empty((0, 5))):
        self.frame_count += 1
        # get predicted locations from existing trackers.
        trks = np.zeros((len(self.trackers), 5))
        to_del = []
        ret = []
        for t, trk in enumerate(trks):
            pos = self.trackers[t].predict()[0]
            trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
            if np.any(np.isnan(pos)):
                to_del.append(t)
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
        for t in reversed(to_del):
            self.trackers.pop(t)
        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks, self.iou_threshold)
         # update matched trackers with assigned detections
        for m in matched:
            self.trackers[m[1]].update(dets[m[0], :])

        # create and initialise new trackers for unmatched detections
        for i in unmatched_dets:
            trk = KalmanBoxTracker(dets[i,:])
            self.trackers.append(trk)
        i = len(self.trackers)
        for trk in reversed(self.trackers):
            d = trk.get_state()[0]
            if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
                ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
            i -= 1
            # remove dead tracklet
            if(trk.time_since_update > self.max_age):
                self.trackers.pop(i)
        if(len(ret)>0):
            return np.concatenate(ret)
        return np.empty((0,5))


Capture on a video

In [24]:
def PlateToVehicle(det_license_plate,det_vehicles):
    lp_x1, lp_y1, lp_x2, lp_y2, lp_score, lp_class_id = det_license_plate
    v_index=None
    for i in len(det_vehicles):
        v_x1, v_y1, v_x2, v_y2, v_score, v_class_id = det_vehicles[i]
        #inside
        if lp_x1 > v_x1 and lp_y1 > v_y1 and lp_x2 < v_x2 and lp_y2 < v_y2:
            v_index=i
            break

    if v_index is not None:
        return det_vehicles[v_index]
    return 0,0,0,0,0

In [25]:
def lpPrep(license_plate):
    lp_gray = cv2.cvtColor(license_plate,cv2.COLOR_BGR2GRAY)
    retval,lp_threshold = cv2.threshold(lp_gray,64,255,cv2.THRESH_BINARY_INV)
    return lp_threshold


In [26]:
def lpRead(license_plate):
    lp_Text='asd012'
    lp_Conf_score = 1
    return lp_Text, lp_Conf_score

In [28]:
df_cols = ['fr_number','v_id', 'v_bbox','lp_bbox','lp_bbox_score','lp','lp_score']
df_rows = []

In [33]:
# load
video_capture = cv2.VideoCapture(testvideo_path)


mot_trckr = MotionTracker()
results = {}
ret = True
indexer=0
while ret:
    if (video_capture.isOpened()== False):
        print("Error opening video stream or file")
    
    ret, frame = video_capture.read()
    
    if ret and indexer<10:
        results[indexer] = {}
        #det
        det_objs = vehicle_det_model(frame)[0]
        vehicles=[]
        for det_obj in det_objs.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = det_obj
            if int(class_id) in vehicle_ids:
                vehicles.append([x1, y1, x2, y2, score])
        
        track_ids = mot_trckr.update(np.asarray(vehicles))

        #plates=[]
        det_lps = plate_det_model(frame)[0]
        for det_lp in det_lps.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = det_lp
            
            #assign license plate to vehicle
            vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2,vehicle_id = PlateToVehicle(det_lp,track_ids)

            #plates.append([x1, y1, x2, y2, score, class_id])

            #crop
            license_plate = frame[int(y1):int(y2),int(x1):int(x2),:]
            prepared = lpPrep(license_plate)
            #cv2.imshow('lp',license_plate)
            #cv2.imshow('lpprep',prepared)
            #cv2.waitKey(0)

            lp_text, lp_text_confscore = lpRead(prepared)
            if lp_text is not None:
                #results[indexer][vehicle_id] = {'vehicle': {'bbox': [vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2]},
                #                                'license_plate': {'bbox': [x1, y1, x2, y2],
                #                                                  'text':lp_text,
                #                                                  'bbox_confscore':score,
                #                                                  'text_confscore':lp_text_confscore
                #                                                 }}
                df_rows.append({'fr_number':indexer,
                                'v_id':class_id,
                                'v_bbox':[vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2],
                                'lp_bbox':[x1, y1, x2, y2],
                                'lp_bbox_score':score,
                                'lp':lp_text,
                                'lp_score':lp_text_confscore})
                
            
    
    indexer+=1
print(results)
#print(plates)

[ WARN:0@3070.038] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@3070.038] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin4 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@3070.039] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@3070.039] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-su

TypeError: list.append() takes no keyword arguments

In [36]:
out = pd.DataFrame(df_rows,columns=df_cols)

In [37]:
out

,fr_number,v_id,v_bbox,lp_bbox,lp_bbox_score,lp,lp_score
0,0,0.0,"[0, 0, 0, 0]","[355.4010314941406, 719.619140625, 461.6276550...",0.467115,asd012,1
1,0,0.0,"[0, 0, 0, 0]","[245.42874145507812, 1164.503662109375, 389.13...",0.335403,asd012,1
2,0,0.0,"[0, 0, 0, 0]","[986.8820190429688, 815.9974365234375, 1114.75...",0.333886,asd012,1
3,0,0.0,"[0, 0, 0, 0]","[1159.7911376953125, 1149.2432861328125, 1293....",0.308768,asd012,1
4,1,0.0,"[0, 0, 0, 0]","[355.40081787109375, 719.6344604492188, 461.58...",0.467120,asd012,1
5,1,0.0,"[0, 0, 0, 0]","[986.96044921875, 816.00048828125, 1114.767578...",0.333503,asd012,1
6,1,0.0,"[0, 0, 0, 0]","[245.3355712890625, 1164.4703369140625, 389.17...",0.332983,asd012,1
7,1,0.0,"[0, 0, 0, 0]","[1159.7852783203125, 1149.1611328125, 1293.970...",0.307354,asd012,1
8,2,0.0,"[0, 0, 0, 0]","[355.51629638671875, 719.6675415039062, 461.61...",0.467570,asd012,1
9,2,0.0,"[0, 0, 0, 0]","[245.30648803710938, 1164.4547119140625, 389.2...",0.336590,asd012,1
